In [2]:
import pandas as pd 


In [6]:
# ///Load the data
df = pd.read_csv(r"C:\Users\arysh\OneDrive\Desktop\luxmark_1\hotel_booking.csv\hotel_booking.csv")
df.head()


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498


In [7]:
df.isnull().sum()


hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [13]:
df.fillna({
    'children': 0,
    'country': 'Unknown',
    'agent': 0,
    'company': 0
}, inplace=True)
df.dtypes


hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [14]:
df['agent'].unique()


array([  0., 304., 240., 303.,  15., 241.,   8., 250., 115.,   5., 175.,
       134., 156., 243., 242.,   3., 105.,  40., 147., 306., 184.,  96.,
         2., 127.,  95., 146.,   9., 177.,   6., 143., 244., 149., 167.,
       300., 171., 305.,  67., 196., 152., 142., 261., 104.,  36.,  26.,
        29., 258., 110.,  71., 181.,  88., 251., 275.,  69., 248., 208.,
       256., 314., 126., 281., 273., 253., 185., 330., 334., 328., 326.,
       321., 324., 313.,  38., 155.,  68., 335., 308., 332.,  94., 348.,
       310., 339., 375.,  66., 327., 387., 298.,  91., 245., 385., 257.,
       393., 168., 405., 249., 315.,  75., 128., 307.,  11., 436.,   1.,
       201., 183., 223., 368., 336., 291., 464., 411., 481.,  10., 154.,
       468., 410., 390., 440., 495., 492., 493., 434.,  57., 531., 420.,
       483., 526., 472., 429.,  16., 446.,  34.,  78., 139., 252., 270.,
        47., 114., 301., 193., 182., 135., 350., 195., 352., 355., 159.,
       363., 384., 360., 331., 367.,  64., 406., 16

In [26]:
df['country'].head() #Check the content of each column

0    PRT
1    PRT
2    GBR
3    GBR
4    GBR
Name: country, dtype: object

In [15]:
df["reservation_status_date"] = pd.to_datetime(df["reservation_status_date"])
df["agent"]= df["agent"].astype(int)
df["company"]=df["company"].astype(int)



In [18]:
#. Check for Duplicates
df.duplicated().sum() # how many duplicate rows?
df.drop_duplicates(inplace=True)


In [24]:
# Existing features
df['total_guests'] = df['adults'] + df['children'] + df['babies']
df['is_family'] = df['children'].apply(lambda x: 1 if x > 0 else 0)
df['stay_length'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

# New useful features

# Average number of guests per night (helps detect overbookings or anomalies)
df['guests_per_night'] = df['total_guests'] / df['stay_length']
df['guests_per_night'] = df['guests_per_night'].replace([float('inf'), -float('inf')], 0)
df['guests_per_night'] = df['guests_per_night'].fillna(0)

# Booking lead time category (binned for analysis)
df['lead_time_category'] = pd.cut(df['lead_time'], 
                                  bins=[-1, 7, 30, 90, 365, 10000], 
                                  labels=['<1 week', '1-4 weeks', '1-3 months', '3-12 months', '1+ year'])

# Was the booking in the high season? (July–August & December)
df['high_season'] = df['arrival_date_month'].isin(['July', 'August', 'December']).astype(int)

# Total number of special requests
df['total_special_requests'] = df['total_of_special_requests']

# Was the booking made for business or leisure?
df['is_business'] = df['distribution_channel'].apply(lambda x: 1 if x.lower() == 'corporate' else 0)

# Weekend stay indicator
df['has_weekend_stay'] = df['stays_in_weekend_nights'].apply(lambda x: 1 if x > 0 else 0)

# Repeat guest flag (already available as is_repeat_guest, but duplicated here as boolean for consistency)
df['repeat_guest_flag'] = df['is_repeated_guest'].apply(lambda x: True if x == 1 else False)
df['guests_per_night'] = df['total_guests'] / df['stay_length'].replace(0, 1)
df['guests_per_night'] = df['guests_per_night'].fillna(0)

df['lead_time_category'] = pd.cut(df['lead_time'], 
                                  bins=[-1, 7, 30, 90, 365, 10000], 
                                  labels=['<1 week', '1-4 weeks', '1-3 months', '3-12 months', '1+ year'])

df['high_season'] = df['arrival_date_month'].isin(['July', 'August', 'December']).astype(int)

df['total_special_requests'] = df['total_of_special_requests']

df['is_business'] = df['distribution_channel'].apply(lambda x: 1 if isinstance(x, str) and x.lower() == 'corporate' else 0)

df['has_weekend_stay'] = (df['stays_in_weekend_nights'] > 0).astype(int)

df['repeat_guest_flag'] = df['is_repeated_guest'] == 1
